In [15]:
pip install opencv-python numpy scikit-image

In [17]:
import os
import cv2
import numpy as np
import random
from pathlib import Path
from tqdm import tqdm

# Paths
input_path = r"C:\Users\sandi\Downloads\idk\fruit_ripe"
output_path = r"C:\Users\sandi\Downloads\augmented"
os.makedirs(output_path, exist_ok=True)

# Safe Augmentations
def rotate(image):
    angle = random.choice([-10, -5, 5, 10])
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    return cv2.warpAffine(image, M, (w, h))

def flip(image):
    return cv2.flip(image, random.choice([0, 1]))  # Horizontal or vertical

def translate(image):
    rows, cols = image.shape[:2]
    tx, ty = random.randint(-20, 20), random.randint(-20, 20)
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(image, M, (cols, rows))

def adjust_brightness(image):
    beta = random.randint(-15, 15)
    return cv2.convertScaleAbs(image, beta=beta)

def jitter_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    s = cv2.add(s, random.randint(-10, 10))
    v = cv2.add(v, random.randint(-10, 10))
    final_hsv = cv2.merge((h, s, v))
    return cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)


# Final Augmentation List
augmentations = [rotate, flip, translate, adjust_brightness, jitter_color]

# Apply to Folder
def augment_folder(input_folder, output_folder, max_images=500):
    image_files = list(Path(input_folder).glob("*"))
    original_images = [cv2.imread(str(img)) for img in image_files if img.suffix.lower() in ['.jpg', '.jpeg', '.png']]
    total_original = len(original_images)
    if total_original == 0:
        return

    os.makedirs(output_folder, exist_ok=True)

    for i in tqdm(range(max_images), desc=f"Augmenting {input_folder}"):
        img = random.choice(original_images)
        aug_img = img.copy()
        applied_augs = random.sample(augmentations, random.randint(1, 3))
        for aug in applied_augs:
            aug_img = aug(aug_img)
        save_path = os.path.join(output_folder, f"img_{i:04d}.jpg")
        cv2.imwrite(save_path, aug_img)

# Process Class Folders
for folder in os.listdir(input_path):
    full_path = os.path.join(input_path, folder)
    if os.path.isdir(full_path):
        output_subfolder = os.path.join(output_path, folder)
        augment_folder(full_path, output_subfolder)


Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\apple_raw: 100%|██████████| 500/500 [00:04<00:00, 122.85it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\apple_ripe: 100%|██████████| 500/500 [00:04<00:00, 108.99it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\apple_rotten: 100%|██████████| 500/500 [00:02<00:00, 231.37it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\banana_raw: 100%|██████████| 500/500 [00:01<00:00, 259.51it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\banana_ripe: 100%|██████████| 500/500 [00:02<00:00, 227.73it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\banana_rotten: 100%|██████████| 500/500 [00:01<00:00, 281.77it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\mango_raw: 100%|██████████| 500/500 [00:01<00:00, 258.94it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\mango_ripe: 100%|██████████| 500/500 [00:01<00:00, 263.52it/s]
Augmenting C:\Users\sandi\Downloads\idk\fruit_ripe\mango_rotten: 100%|██████████| 50